**How To**

1. Jalankan semua cell mulai dari import package, dan jalankan satu persatu atau bisa semuanya, atau bisa menggunakan shortcut CTRL+F9 untuk menjalankan semua, jika ingin menjalankan lagi run cell yang skrip GUI saja tanpa mengulang semua skrip.
2. Ketika program sudah terbuka tinggal pilih media mana yang akan kita scrap.
3. Tunggu program menjalankan scrapping data di rss media.
4. Tampilan yang tersedia merupakan kata yang sering dipakai berupa wordcloud, grafik bar frekuensi, dan tabel berapa kali kata yang dipakai.

In [7]:
# 1. Instalasi sastrawi
!pip install Sastrawi feedparser wordcloud gradio --quiet

In [3]:
# 2. Import Library
import feedparser
import re
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from matplotlib.colors import ListedColormap
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import gradio as gr
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:

# 3. Daftar RSS Berita

rss_dict = {
    "Google News (ID)": "https://news.google.com/rss?hl=id&gl=ID&ceid=ID:id",
   # "Detik - Nasional": "https://rss.detik.com/index.php/nasional",
    "Antara News - Nasional": "https://www.antaranews.com/rss",
    "Media Indonesia - Nasional": "https://mediaindonesia.com/feed",
    "Kontan - Nasional": "https://www.kontan.co.id/rss",
    "Republika - Nasional": "http://www.republika.co.id/rss",
    "CNBC - Nasional": "https://www.cnbcindonesia.com/rss",
  #  "Kompas - Nasional": "https://nasional.kompas.com/rss",
    "BBC - World": "http://feeds.bbci.co.uk/news/world/rss.xml",
    "CNN - World": "http://rss.cnn.com/rss/edition_world.rss",
    "Reuters - World": "http://feeds.reuters.com/Reuters/worldNews"
}

In [5]:

# 4. Fungsi Analisis Berita

def analisis_berita(kategori):
    url = rss_dict.get(kategori)
    feed = feedparser.parse(url)
    judul_berita = [entry.title for entry in feed.entries]

    if len(judul_berita) == 0:
        return "❌ Tidak ada berita ditemukan."

    konten = ' '.join(judul_berita).lower()
    konten = re.sub(r'[^a-zA-Z\s]', '', konten)

    kata_kata = konten.split()

    # Tentukan apakah sumber Indonesia atau bukan
    is_indonesia = 'Google News' in kategori or 'Detik' in kategori or 'Kompas' in kategori or 'Antara' in kategori or 'Media Indonesia' in kategori or 'Kontan' in kategori or 'Republika' in kategori or 'CNBC' in kategori

    if is_indonesia:
        # Stopword & Stemming Bahasa Indonesia
        stop_factory = StopWordRemoverFactory()
        stopwords_id = set(stop_factory.get_stop_words())
        stem_factory = StemmerFactory()
        stemmer = stem_factory.create_stemmer()

        kata_bersih = [kata for kata in kata_kata if kata not in stopwords_id and kata.strip() != '']
        kata_stemmed = [stemmer.stem(kata) for kata in kata_bersih]
        final_kata = kata_stemmed
    else:
        # Stopwords Bahasa Inggris
        stopwords_en = set(stopwords.words('english'))
        final_kata = [kata for kata in kata_kata if kata not in stopwords_en and kata.strip() != '']

    if len(final_kata) == 0:
        return "❌ Tidak ada kata yang bisa dianalisis."

    # WordCloud
    warna = ListedColormap(['orange', 'green', 'red', 'blue'])
    wordcloud = WordCloud(background_color='white', max_words=100, colormap=warna)
    wordcloud_img = wordcloud.generate(' '.join(final_kata)).to_image()

    # Grafik batang
    frekuensi = pd.Series(final_kata).value_counts().head(15)
    plt.figure(figsize=(8, 4))
    frekuensi.plot(kind='bar', color='skyblue')
    plt.title(f'Kata Umum di Judul Berita: {kategori}')
    plt.xlabel('Kata')
    plt.ylabel('Jumlah')
    plt.xticks(rotation=45)
    plt.grid(axis='y')
    plt.tight_layout()
    plt.savefig("bar_plot.png")
    plt.close()

    # Tabel frekuensi kata
    tabel_frekuensi = pd.DataFrame({'Kata': frekuensi.index, 'Jumlah': frekuensi.values})

    return wordcloud_img, "bar_plot.png", tabel_frekuensi

In [6]:

# 5. GUI

gr.Interface(
    fn=analisis_berita,
    inputs=gr.Dropdown(choices=list(rss_dict.keys()), label="Pilih Sumber Berita"),
    outputs=[
        gr.Image(type="pil", label="WordCloud"),
        gr.Image(type="filepath", label="Grafik Frekuensi"),
        gr.Dataframe(label="Tabel Kata Paling Umum")
    ],
    title="📊 Scrap Berita RSS (Nasional & Internasional)",
    description="judul berita tanpa kata sambung (stopwords) seperti the, of, dan, yang, dll.",
    theme="default"
).launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://80109e160fe51ab9cf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Improvement**

1. Kata yang discrap merupakan Headline berita dan tidak termasuk isi berita tsb.
2. Beberapa media tidak menyediakan RSS. misalnya, media nasional detik.
3. Bisa dipakai beberapa fitur untuk menambah kata yang discrap secara sepesifik.
4. Masih adanya bug seperti kata bulu tangkis karena dalam KBBI harusnya disambung, tetapi di skrip ini belum ada stopword untuk hal tersebut, dan masih menggunakan stopword kata sambung. seperti, dan, yang, lain, ke, dll.